# Hello Evolution: A baseline GA

This is a demonstration of an GA for automatic programming of a Synthesizer. We assume a synthesizer with the following parameters

- `osc_1` : The shape of the first oscillator (eg Sine, Sawtooth, Triangle, Square etc)
- `amp_1` : The amplitude of the first oscillator in the range 0.3-0.7.
- `osc_2` : The shape of the second oscillator (eg Sine, Sawtooth, Triangle, Square etc)
- `amp_2` : The amplitude of the second oscillator in the range 0.3-0.7.

It holds that 

`amp_2` = 1-`amp_1`

so in future experiments, the `amp_2` parameter can be omitted.

First let's import all necessary modules. For this experiment we use the [deap](https://github.com/DEAP/deap) library.

In [1]:
from deap import creator, base, tools, algorithms
import numpy as np
import random
import synth
import librosa
import multiprocessing
import time

Now let's define the parameters that we are going to use and the osc shape values.

In [2]:
labels = ['osc_1', 'amp_1', 'osc_2', 'amp_2']
SHAPES = list(synth.osc_1_options.keys())
print(SHAPES)

['Sine', 'Triangle', 'Square', 'SquareH', 'Sawtooth', 'Pulse', 'Semicircle']


Now let's define the target sound. The ground truth parameters are:
```
osc_1 = 'Sine'
amp_1 = 0.4
osc_2 = 'Sawtooth'
amp_2 = 0.6
```

In [3]:
mysynth = synth.Synth()
target_features = mysynth.set_parameters(osc_1='Sine',
                                         amp_1=0.4,
                                         osc_2='Sawtooth',
                                         amp_2=0.6)
soundarray = mysynth.get_sound_array()

Time for the definition of the feature extraction, the mate and mutate operators and the evaluation function.

In [4]:
def features(sound_array):
    sr = 44100
    centroid = librosa.feature.spectral_centroid(sound_array, sr)[0]
    bandwidth = librosa.feature.spectral_bandwidth(sound_array, sr)[0]
    flatness = librosa.feature.spectral_flatness(sound_array)[0]
    rolloff = librosa.feature.spectral_rolloff(sound_array)[0]
    result = np.array([centroid, bandwidth, flatness, rolloff])
    return result.flatten()

# Target feature vector
target_f = features(soundarray)

def custom_evaluate(params):
    """Sum of squared error evaluation function"""
    mysynth = synth.Synth()
    params = dict(zip(labels, params))
    mysynth.set_parameters(**params)
    soundarray = mysynth.get_sound_array()
    f_vector = features(soundarray)
    return np.sum(np.square(f_vector - target_f)),

def custom_mate(ind1, ind2):
    """Mating function. 
    Selects one of the parameters and swaps them.
    """
    pos = random.choice(np.arange(0,4))
    ind1[pos], ind2[pos] = ind2[pos], ind1[pos]
    return ind1, ind2

def custom_mutate(individual):
    """Mutation function. 
    Randomly modifies one of the parameters
    """
    i = random.choice([0,1,2,3])
    if i == 0 or i == 2:
        individual[i] = random.choice(SHAPES)
    else:
        individual[i] = np.random.uniform(low=0.3, high=0.7, size=(1,))[0]
    
    return individual,

Now let's define the GA algorithm with the help of DEAP

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize Fitness (negative weight)
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_shape", lambda : random.choice(SHAPES))
toolbox.register("attr_amp", lambda : np.random.uniform(low=0.3, high=0.7, size=(1,))[0])
toolbox.register("individual",
                 tools.initCycle,
                 creator.Individual,
                 (toolbox.attr_shape, toolbox.attr_amp),
                 2)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", custom_evaluate) 
toolbox.register("mate", custom_mate)
toolbox.register("mutate", custom_mutate)
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection
toolbox.register("map", multiprocessing.Pool().map)  # Make it parallel AND FAST

Time to run the algorithm. We pick 10 generations of 30 individuals.
But first, let's see the initial generation.

In [6]:
population = toolbox.population(n=30)
fits = toolbox.map(toolbox.evaluate, population)
print('Mean error of the initial population', np.mean(list(fits)))
population

Mean error of the initial population 259897209.1480775


[['Semicircle', 0.6781042167639075, 'Sawtooth', 0.44867418100948675],
 ['Triangle', 0.5001223536065843, 'Sawtooth', 0.3769479095101146],
 ['Triangle', 0.303994657203925, 'Sine', 0.4160456782950453],
 ['Square', 0.4111836493750859, 'Triangle', 0.3381852552166252],
 ['Square', 0.55486317622704, 'Square', 0.4399174576381645],
 ['Triangle', 0.31055370339893534, 'Triangle', 0.6473926947809505],
 ['Sawtooth', 0.5310391654236788, 'Semicircle', 0.6766539518061502],
 ['Sawtooth', 0.39571601640817705, 'Triangle', 0.6464075042793473],
 ['SquareH', 0.6627908085486699, 'Triangle', 0.3304517080722218],
 ['SquareH', 0.5528311008104229, 'SquareH', 0.5767326591847699],
 ['Pulse', 0.5582335166059699, 'Pulse', 0.3072153207345294],
 ['Sawtooth', 0.3160622584920359, 'Pulse', 0.6905961144348713],
 ['Sawtooth', 0.602721008025232, 'Triangle', 0.5965121461127091],
 ['SquareH', 0.5951751384140711, 'Sine', 0.3758527380938185],
 ['Triangle', 0.5894732475232007, 'Pulse', 0.4841392363147976],
 ['Square', 0.60174916

In [7]:
start = time.time()
NGEN=10
for gen in range(1, NGEN+1):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = list(toolbox.map(toolbox.evaluate, offspring))
    mean_fit = 0
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        mean_fit += fit[0]/len(population)
    print('Mean error generation', gen, ':', np.mean(fits),
          'Error of best individual', gen, ':', np.min(fits))
    population = toolbox.select(offspring, k=len(population))
print('Total runtime:', time.time()-start)

Mean error generation 1 : 260799106.152245 Error of best individual 1 : 93288.86359254728
Mean error generation 2 : 58394451.84379439 Error of best individual 2 : 93288.86359254728
Mean error generation 3 : 62152556.99687283 Error of best individual 3 : 93288.86359254728
Mean error generation 4 : 3330953.353934851 Error of best individual 4 : 93288.86359254728
Mean error generation 5 : 497746.46283773717 Error of best individual 5 : 93288.86359254728
Mean error generation 6 : 111249771.92731032 Error of best individual 6 : 93288.86359254728
Mean error generation 7 : 1064352.6292607742 Error of best individual 7 : 57886.54431449386
Mean error generation 8 : 62277992.660083994 Error of best individual 8 : 35308.56725205307
Mean error generation 9 : 1653378.751280148 Error of best individual 9 : 35308.56725205307
Mean error generation 10 : 1060028.671644334 Error of best individual 10 : 35308.56725205307
Total runtime: 21.827521800994873


The two best results are:

In [8]:
tools.selBest(population, k=2)

[['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185]]

In [9]:
population

[['Sawtooth', 0.5951751384140711, 'Sine', 0.3810769849928337],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5880284388866324, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3810769849928337],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3810769849928337],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3810769849928337],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3810769849928337],
 ['Sawtooth', 0.5951751384140711, 'Sine', 0.3758527380938185],
 ['Sawtooth', 0.5461182351802976, 'Sine', 0.37585273809